In [46]:
import cv2
import numpy as np
import mediapipe as mp
from collections import deque
import math
import os
import time
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.models import load_model

# Ensure drawings folder exists
os.makedirs("drawings", exist_ok=True)

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Drawing variables
colors = [(255, 0, 255), (0, 255, 0), (0, 0, 255)]
tools = ['Pen1', 'Pen2', 'Pen3', 'Eraser', 'Rect', 'Circle', 'Line', 'Arrow', 'Ellipse']
draw_color = colors[0]
tool = 'Pen1'
brush_thickness = 10
fill_shapes = False

# Canvas
xp, yp = 0, 0
img_canvas = np.zeros((480, 640, 3), np.uint8)
history = deque()
redo_stack = deque()
last_save_time = 0

# CNN Digit Recognition Model
model = load_model("digit_recognition_model.h5")

def preprocess_image(img):
    img = cv2.resize(img, (28, 28))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.reshape(28, 28, 1)
    img = img / 255.0
    return img

def predict_digit(img):
    img = preprocess_image(img)
    img = np.expand_dims(img, axis=0)
    predictions = model.predict(img)
    digit = np.argmax(predictions)
    confidence = predictions[0][digit]
    return digit, confidence

# Helper functions
def add_to_history(canvas):
    history.append(canvas.copy())
    if len(history) > 20:
        history.popleft()
    redo_stack.clear()

def draw_buttons(frame):
    for i, t in enumerate(tools):
        x1, y1, x2, y2 = i * 70 + 10, 10, i * 70 + 70, 60
        cv2.rectangle(frame, (x1, y1), (x2, y2), (50, 50, 50), -1)
        color = colors[i] if i < 3 else (255, 255, 255)
        if tool == t:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 3)
        cv2.putText(frame, t, (x1 + 5, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

def get_distance(p1, p2):
    return int(math.hypot(p1[0] - p2[0], p1[1] - p2[1]))

def save_drawing(canvas):
    filename = f"drawings/{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    cv2.imwrite(filename, canvas)
    print(f"Drawing saved: {filename}")
    return time.time()

# Webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)
    draw_buttons(frame)

    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            lm = handLms.landmark
            h, w, _ = frame.shape
            coords = [(int(pt.x * w), int(pt.y * h)) for pt in lm]
            x1, y1 = coords[8]
            x2, y2 = coords[12]
            thumb_tip = coords[4]

            fingers = [lm[i].y < lm[i - 2].y for i in [8, 12, 16, 20]]
            total_fingers = fingers.count(True)

            if total_fingers == 5:
                pass  # Reserved for future use

            elif total_fingers == 3:
                img_canvas = np.zeros((480, 640, 3), np.uint8)
                history.clear()
                redo_stack.clear()
                cv2.putText(frame, "Canvas Cleared", (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            elif total_fingers >= 4:
                if time.time() - last_save_time > 2:
                    last_save_time = save_drawing(img_canvas)
                    cv2.putText(frame, "Drawing Saved", (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                                (0, 255, 0), 2)

            elif fingers[0] and fingers[1]:  # Selection Mode
                xp, yp = 0, 0
                for i, t in enumerate(tools):
                    if 10 + i * 70 < x1 < 70 + i * 70 and 10 < y1 < 60:
                        tool = tools[i]
                        if tool.startswith('Pen'):
                            draw_color = colors[i]
                        break
                cv2.rectangle(frame, (x1, y1 - 20), (x2, y2 + 20), draw_color, cv2.FILLED)

            elif fingers[0] and not fingers[1]:  # Drawing Mode
                if xp == 0 and yp == 0:
                    xp, yp = x1, y1
                    add_to_history(img_canvas)

                thickness = get_distance(thumb_tip, coords[8]) // 2
                current_thickness = max(1, thickness) if tool.startswith('Pen') else brush_thickness

                if tool == 'Eraser':
                    cv2.line(frame, (xp, yp), (x1, y1), (0, 0, 0), 50)
                    cv2.line(img_canvas, (xp, yp), (x1, y1), (0, 0, 0), 50)

                elif tool.startswith('Pen'):
                    cv2.line(frame, (xp, yp), (x1, y1), draw_color, current_thickness)
                    cv2.line(img_canvas, (xp, yp), (x1, y1), draw_color, current_thickness)

                else:
                    img_canvas[:] = history[-1].copy() if history else img_canvas
                    shape_thickness = -1 if fill_shapes else brush_thickness

                    if tool == 'Rect':
                        cv2.rectangle(img_canvas, (xp, yp), (x1, y1), draw_color, shape_thickness)
                    elif tool == 'Circle':
                        center = ((xp + x1) // 2, (yp + y1) // 2)
                        radius = get_distance((xp, yp), (x1, y1)) // 2
                        cv2.circle(img_canvas, center, radius, draw_color, shape_thickness)

                        digit, confidence = predict_digit(img_canvas)
                        cv2.putText(frame, f"Digit: {digit} ({confidence:.2f})", (x1, y1 - 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                    elif tool == 'Line':
                        cv2.line(img_canvas, (xp, yp), (x1, y1), draw_color, brush_thickness)
                    elif tool == 'Arrow':
                        cv2.arrowedLine(img_canvas, (xp, yp), (x1, y1), draw_color, brush_thickness)
                    elif tool == 'Ellipse':
                        center = ((xp + x1) // 2, (yp + y1) // 2)
                        axes = (abs(x1 - xp) // 2, abs(y1 - yp) // 2)
                        cv2.ellipse(img_canvas, center, axes, 0, 0, 360, draw_color, shape_thickness)

                xp, yp = x1, y1

            mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)

    gray_canvas = cv2.cvtColor(img_canvas, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray_canvas, 50, 255, cv2.THRESH_BINARY)
    inv = cv2.bitwise_not(mask)
    frame = cv2.bitwise_and(frame, frame, mask=inv)
    frame = cv2.bitwise_or(frame, img_canvas)

    cv2.imshow("Virtual Whiteboard", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('z') and history:
        redo_stack.append(img_canvas.copy())
        img_canvas = history.pop()
    elif key == ord('y') and redo_stack:
        history.append(img_canvas.copy())
        img_canvas = redo_stack.pop()
    elif key == 27:
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Drawing saved: drawings/20250513_185757.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1